In [1]:
# Dependencies
import json
from pprint import pprint
from datetime import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tweepy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from config import (consumer_key, consumer_secret,
                    access_token, access_token_secret)

In [2]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), 
                 wait_on_rate_limit=False, wait_on_rate_limit_notify=False)

# Initialize Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

In [3]:
# Movie csv path
csv_path = 'movie_csv/moviesFinal.csv'

# Read csv
movie_df = pd.read_csv(csv_path)
movie_df

,title,genre,release,value,budget,opening,domestic,worldwide,ratio
0,Split,Horror,20170120,70.00,5000000,40010975,138141585,278449984,8.002195
1,xXx: Return of Xander Cage,Action,20170120,46.33,85000000,20130142,44898413,345114933,0.236825
2,A Dog’s Purpose,Drama,20170127,48.33,25000000,18222810,64321890,205272612,0.728912
3,Resident Evil: The Final Chapter,Action,20170127,47.00,40000000,13601682,26844692,307353906,0.340042
4,The Bye Bye Man,Thriller/Suspense,20170113,34.33,7400000,13501349,22395806,25863405,1.824507
5,Monster Trucks,Adventure,20170113,43.00,125000000,10950705,33370166,61681268,0.087606
6,The Founder,Drama,20170120,73.67,15000000,3404102,12786243,18301418,0.226940
7,A Monster Calls,Drama,20170106,79.00,43000000,30909,3740823,46422066,0.000719
8,Emily,Drama,20170127,70.00,27000,1164,3547,3547,0.043111
9,The Lego Batman Movie,Adventure,20170210,79.67,80000000,53003468,175750384,310940997,0.662543


In [18]:
# Target user
# search_term = "Split"

# # "Real Person" Filters
# min_tweets = 5
# max_tweets = 10000
# max_followers = 2500
# max_following = 2500
# lang = "en"

# # Use filters to check if user meets conditions
# if (tweet["user"]["followers_count"] < max_followers and
#     tweet["user"]["statuses_count"] > min_tweets and
#     tweet["user"]["statuses_count"] < max_tweets and
#     tweet["user"]["friends_count"] < max_following and
#     tweet["user"]["lang"] == lang):

# try:

# Create list of dictionaries
sentiment = []

for title in movie_df['title']:
    
    # Assign title as search term
    search_term = title
    
    # Create variable for holding the oldest tweet
    oldest_tweet = None

    # List to hold average compound values for each movie
    compound_list = []

    # Loop through 5 times
    for x in range(1):

        # Get all tweets from home feed (for each page specified)
        public_tweets = api.search(search_term, 
                                   count=100, 
                                   lang='en',
                                   result_type="recent",
                                   max_id=oldest_tweet)

        # Loop through all tweets
        for tweet in public_tweets['statuses']:
               
            # Grab tweet data
            tweet_text = tweet['text']
            
            # Run Vader Analysis on each tweet
            results = analyzer.polarity_scores(tweet["text"])
            compound = results["compound"]
            
            # Append compound value to list
            compound_list.append(compound)
            
            # Reassign the the oldest tweet (i.e. the max_id)
            oldest_tweet = int(tweet["id_str"])
        
            # Subtract 1 so the previous oldest isn't included
            # in the new search
            oldest_tweet -= 1

    # Store average 
    tweet_dict = {"Movie Title": search_term, 
                  "Compound": np.mean(compound_list), 
                  "Tweet Count": len(compound_list)}
    
    # Append tweet data to sentiment list
    sentiment.append(tweet_dict)

# except:
#     print("You exceeded the rate limit!")
#     raise

C:\Users\Justin\Anaconda3\envs\PythonData\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Justin\Anaconda3\envs\PythonData\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [19]:
movie_sent_df = pd.DataFrame(sentiment)
movie_sent_df

,Compound,Movie Title,Tweet Count
0,0.152004,Split,100
1,-0.135682,xXx: Return of Xander Cage,100
2,-0.003925,A Dog’s Purpose,100
3,-0.568297,Resident Evil: The Final Chapter,94
4,0.261678,The Bye Bye Man,100
5,0.100423,Monster Trucks,100
6,0.059512,The Founder,100
7,0.290893,A Monster Calls,100
8,0.046960,Emily,100
9,0.256969,The Lego Batman Movie,35
